In [4]:
import sqlite3
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from imblearn.metrics import classification_report_imbalanced
from sklearn.model_selection import train_test_split
import os

In [2]:
db = r'C:/Users/jonat/UO_Bootcamp/Group_project/git_Water_Quality_Analysis/Water_Quality_Analysis/Database/database.sqlite3'
# Connect to SQLite database
conn = sqlite3.connect(db)
  
# Create cursor object
cursor = conn.cursor()

#Read in the Data from the DB
df = pd.read_sql_query("SELECT * FROM Census_Data INNER JOIN Contaminant_Summary on Census_Data.county_FIPS = Contaminant_Summary.county_FIPS",conn)

In [12]:
#Get the target binary data from the .csv file that was generated in the Priority_algo_dev.ipynb
target = pd.read_csv(os.path.join('..', '..', 'Priority_Algo_dev', 'new_priority.csv'))
target

,Unnamed: 0,New_Priority
0,0,0
1,1,1
2,2,1
3,3,1
4,4,0
...,...,...
877,877,0
878,878,0
879,879,0
880,880,1


In [13]:
target = target.drop(columns=['Unnamed: 0'])
target

,New_Priority
0,0
1,1
2,1
3,1
4,0
...,...
877,0
878,0
879,0
880,1


In [14]:
#The problem is the imbalanced data and this will need to be addressed
target.New_Priority.value_counts()

0    711
1    171
Name: New_Priority, dtype: int64

In [15]:
df.sample(20)

,county_FIPS,Geographic_Area_Name,County,GEOID,Total_Population,White,Black,Native,Asian,Pacific_Islander,...,Shannon_Race_DI,Shannon_Ethnic_DI,Gini_Index,County_FIPS,Num_Contaminants,Sum_Population_Served,Sum_ContaminantFactor,Min_Contaminant_Factor,Max_Contaminant_Factor,Avg_Contaminant_Factor
855,6027,"Inyo County, California",Inyo County,0500000US06027,19016,11752,98,2474,283,18,...,1.496515,0.540880,0.4277,6027,9,2131,16622,0,969,1846.89
444,20007,"Barber County, Kansas",Barber County,0500000US20007,4228,3857,9,35,17,6,...,0.529199,0.162555,0.4606,20007,6,3559,10378,1045,690,1729.67
97,46079,"Lake County, South Dakota",Lake County,0500000US46079,11059,10138,108,129,72,13,...,0.568949,0.212404,0.4193,46079,3,3275,1710,441,658,570.00
213,49023,"Juab County, Utah",Juab County,0500000US49023,11786,10972,19,99,35,45,...,0.489140,0.194675,0.3684,49023,1,851,46,46,46,46.00
794,42101,"Philadelphia County, Pennsylvania",Philadelphia County,0500000US42101,1603797,582636,630463,6213,133554,933,...,1.647785,0.420222,0.5170,42101,2,69144,1411,1117,294,705.50
359,17003,"Alexander County, Illinois",Alexander County,0500000US17003,5240,3311,1618,25,7,1,...,0.926251,0.080584,0.4722,17003,5,5568,2372,265,686,474.40
439,17201,"Winnebago County, Illinois",Winnebago County,0500000US17201,285350,191124,39160,1633,8040,85,...,1.352421,0.411293,0.4486,17201,15,7965,3503,105,84,233.53
770,42053,"Forest County, Pennsylvania",Forest County,0500000US42053,6973,5296,1238,10,20,1,...,0.890408,0.187538,0.3909,42053,3,5380,1822,110,899,607.33
771,42055,"Franklin County, Pennsylvania",Franklin County,0500000US42055,155932,134524,5339,489,1443,53,...,0.768164,0.251123,0.4102,42055,14,8108,3984,111,99,284.57
15,38035,"Grand Forks County, North Dakota",Grand Forks County,0500000US38035,73170,60056,3066,1811,2496,60,...,0.892241,0.198455,0.4603,38035,6,12238,4142,78,1627,690.33


In [16]:
df.dtypes

county_FIPS                 int64
Geographic_Area_Name       object
County                     object
GEOID                      object
Total_Population            int64
White                       int64
Black                       int64
Native                      int64
Asian                       int64
Pacific_Islander            int64
Other                       int64
Two_or_more_Races           int64
Hispanic                    int64
Not_Hispanic                int64
Not_White                   int64
pct_White                 float64
pct_Black                 float64
pct_Native                float64
pct_Asian                 float64
pct_Pacific_Islander      float64
pct_Other                 float64
pct_Not_White             float64
pct_Hispanic              float64
pct_Not_Hispanic          float64
pct_Two_or_more_Races     float64
Simpson_Race_DI           float64
Simpson_Ethnic_DI         float64
Shannon_Race_DI           float64
Shannon_Ethnic_DI         float64
Gini_Index    

In [17]:
df.columns

Index(['county_FIPS', 'Geographic_Area_Name', 'County', 'GEOID',
       'Total_Population', 'White', 'Black', 'Native', 'Asian',
       'Pacific_Islander', 'Other', 'Two_or_more_Races', 'Hispanic',
       'Not_Hispanic', 'Not_White', 'pct_White', 'pct_Black', 'pct_Native',
       'pct_Asian', 'pct_Pacific_Islander', 'pct_Other', 'pct_Not_White',
       'pct_Hispanic', 'pct_Not_Hispanic', 'pct_Two_or_more_Races',
       'Simpson_Race_DI', 'Simpson_Ethnic_DI', 'Shannon_Race_DI',
       'Shannon_Ethnic_DI', 'Gini_Index', 'County_FIPS', 'Num_Contaminants',
       'Sum_Population_Served', 'Sum_ContaminantFactor',
       'Min_Contaminant_Factor', 'Max_Contaminant_Factor',
       'Avg_Contaminant_Factor'],
      dtype='object')

## Feature Selection

In [18]:
df_model = df.drop(columns=['county_FIPS', 
                            'Geographic_Area_Name', 
                            'County', 'GEOID',
                            'Total_Population',
                            'White', 
                            'Black', 
                            'Native', 
                            'Asian',
                            'Pacific_Islander', 
                            'Other', 
                            'Two_or_more_Races', 
                            'Hispanic',
                            'Not_Hispanic', 
                            'Not_White',
                            'pct_White', 
                            'pct_Black', 
                            'pct_Native',
                            'pct_Asian', 
                            'pct_Pacific_Islander', 
                            'pct_Other', 
                            'pct_Not_White',
                            'pct_Hispanic', 
                            'pct_Not_Hispanic', 
                            'pct_Two_or_more_Races',
                            'County_FIPS',
                            'Sum_Population_Served',
                            'Min_Contaminant_Factor', 
                            'Max_Contaminant_Factor',
                            ])

In [19]:
#Check to make sure the data types don't need fixing
df_model.dtypes

Simpson_Race_DI           float64
Simpson_Ethnic_DI         float64
Shannon_Race_DI           float64
Shannon_Ethnic_DI         float64
Gini_Index                float64
Num_Contaminants            int64
Sum_ContaminantFactor       int64
Avg_Contaminant_Factor    float64
dtype: object

In [20]:
#Check for Nan even though cleaning scripts should have excluded them by this stage
df_model.isna().sum()

Simpson_Race_DI           0
Simpson_Ethnic_DI         0
Shannon_Race_DI           0
Shannon_Ethnic_DI         0
Gini_Index                0
Num_Contaminants          0
Sum_ContaminantFactor     0
Avg_Contaminant_Factor    0
dtype: int64

## Split the data into training and test data

In [21]:
# Create our features
X = df_model
# Create our target
y = target

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [23]:
#Check the imbalance in the training set
y_train.value_counts()

New_Priority
0               533
1               128
dtype: int64

## Ensemble Learners

### Balanced Random Forest Classifier

In [24]:
# Resample the training data with the BalancedRandomForestClassifier
from imblearn.ensemble import BalancedRandomForestClassifier
brf_model = BalancedRandomForestClassifier(n_estimators=100, random_state=1) 
brf_model.fit(X_train,y_train)

BalancedRandomForestClassifier(random_state=1)

In [25]:
# Calculated the balanced accuracy score
y_pred = brf_model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9711262085184218

In [26]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual High-Priority", "Actual Low-Priority"],
    columns=["Predicted High-Priority", "Predicted Low-Priority"]
)

# Displaying results
display(cm_df)

,Predicted High-Priority,Predicted Low-Priority
Actual High-Priority,176,2
Actual Low-Priority,2,41


In [27]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.99      0.95      0.99      0.97      0.95       178
          1       0.95      0.95      0.99      0.95      0.97      0.94        43

avg / total       0.98      0.98      0.96      0.98      0.97      0.94       221



In [28]:
# List the features sorted in descending order by feature importance
sorted(zip(brf_model.feature_importances_, X.columns), reverse=True)

[(0.25164742230451703, 'Simpson_Ethnic_DI'),
 (0.2301124354492361, 'Shannon_Ethnic_DI'),
 (0.17001837169753242, 'Sum_ContaminantFactor'),
 (0.16444440667019664, 'Shannon_Race_DI'),
 (0.08158584538291756, 'Simpson_Race_DI'),
 (0.06033022445854257, 'Num_Contaminants'),
 (0.027388064347540847, 'Avg_Contaminant_Factor'),
 (0.014473229689516772, 'Gini_Index')]

### Easy Ensemble AdaBoost Classifier

In [29]:
# Train the EasyEnsembleClassifier
from imblearn.ensemble import EasyEnsembleClassifier 
eec = EasyEnsembleClassifier(n_estimators=100, random_state=1)
eec.fit(X_train,y_train)

EasyEnsembleClassifier(n_estimators=100, random_state=1)

In [30]:
# Calculated the balanced accuracy score
y_pred = eec.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.977136137967076

In [31]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual High-Priority", "Actual Low-Priority"],
    columns=["Predicted High-Priority", "Predicted Low-Priority"]
)

# Displaying results
display(cm_df)

,Predicted High-Priority,Predicted Low-Priority
Actual High-Priority,174,4
Actual Low-Priority,1,42


In [32]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.99      0.98      0.98      0.99      0.98      0.95       178
          1       0.91      0.98      0.98      0.94      0.98      0.95        43

avg / total       0.98      0.98      0.98      0.98      0.98      0.95       221



### Naive Random Oversampling

In [33]:
# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
# Resample the training data with the RandomOversampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

y_resampled.value_counts()

New_Priority
0               533
1               533
dtype: int64

In [34]:
from sklearn.linear_model import LogisticRegression
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [35]:
# make predictions
y_pred = model.predict(X_test)

In [36]:
from sklearn.metrics import balanced_accuracy_score
#Calculate the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

0.5682649594983016

In [37]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual High-Priority", "Actual Low-Priority"],
    columns=["Predicted High-Priority", "Predicted Low-Priority"]
)

# Displaying results
display(cm_df)

,Predicted High-Priority,Predicted Low-Priority
Actual High-Priority,45,133
Actual Low-Priority,5,38


In [38]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.90      0.25      0.88      0.39      0.47      0.21       178
          1       0.22      0.88      0.25      0.36      0.47      0.24        43

avg / total       0.77      0.38      0.76      0.39      0.47      0.21       221



### SMOTE Oversampling

In [39]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE
X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(
    X_train, y_train
)
y_resampled.value_counts()

New_Priority
0               533
1               533
dtype: int64

In [40]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [41]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.5730990331852626

In [42]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual High-Priority", "Actual Low-Priority"],
    columns=["Predicted High-Priority", "Predicted Low-Priority"]
)

# Displaying results
display(cm_df)

,Predicted High-Priority,Predicted Low-Priority
Actual High-Priority,55,123
Actual Low-Priority,7,36


In [43]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.89      0.31      0.84      0.46      0.51      0.25       178
          1       0.23      0.84      0.31      0.36      0.51      0.27        43

avg / total       0.76      0.41      0.73      0.44      0.51      0.25       221



## Undersampling


In [44]:
# Resample the data using the ClusterCentroids resampler
from imblearn.under_sampling import ClusterCentroids
cc = ClusterCentroids(random_state=1)
X_resampled, y_resampled = cc.fit_resample(X_train, y_train)
Counter(y_resampled)

Counter({'New_Priority': 1})

In [45]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [46]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7712960543506664

In [47]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual High-Priority", "Actual Low-Priority"],
    columns=["Predicted High-Priority", "Predicted Low-Priority"]
)

# Displaying results
display(cm_df)

,Predicted High-Priority,Predicted Low-Priority
Actual High-Priority,109,69
Actual Low-Priority,3,40


In [48]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.97      0.61      0.93      0.75      0.75      0.55       178
          1       0.37      0.93      0.61      0.53      0.75      0.59        43

avg / total       0.86      0.67      0.87      0.71      0.75      0.56       221



## Combination (Over and Under) Sampling


In [49]:
# Resample the training data with SMOTEENN
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=1)
X_resampled, y_resampled = smote_enn.fit_resample(X_train, y_train)
y_resampled.value_counts()

New_Priority
0               365
1               354
dtype: int64

In [50]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)
model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [51]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)


0.8696106610922394

In [52]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual High-Priority", "Actual Low-Priority"],
    columns=["Predicted High-Priority", "Predicted Low-Priority"]
)

# Displaying results
display(cm_df)

,Predicted High-Priority,Predicted Low-Priority
Actual High-Priority,144,34
Actual Low-Priority,3,40


In [53]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.98      0.81      0.93      0.89      0.87      0.74       178
          1       0.54      0.93      0.81      0.68      0.87      0.76        43

avg / total       0.89      0.83      0.91      0.85      0.87      0.75       221



## Naive Bayes Classifier

In [54]:
from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()
gnb.fit(X_train, y_train)

GaussianNB()

In [55]:
# Calculated the balanced accuracy score
y_pred = gnb.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.7227593415207735

## XGBoost

In [56]:
from xgboost import XGBClassifier
model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss')
model.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric='mlogloss', gamma=0, gpu_id=-1,
              grow_policy='depthwise', importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_bin=256, max_cat_to_onehot=4, max_delta_step=0, max_depth=6,
              max_leaves=0, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=0,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, ...)

In [57]:
# Calculated the balanced accuracy score
y_pred = model.predict(X_test)
balanced_accuracy_score(y_test, y_pred)

0.9418604651162791

In [58]:
# Display the confusion matrix
cm = confusion_matrix(y_test, y_pred)

cm_df = pd.DataFrame(
    cm, index=["Actual High-Priority", "Actual Low-Priority"],
    columns=["Predicted High-Priority", "Predicted Low-Priority"]
)

# Displaying results
display(cm_df)

,Predicted High-Priority,Predicted Low-Priority
Actual High-Priority,178,0
Actual Low-Priority,5,38
